In [6]:
import yfinance as yf
import pandas as pd
import configparser
import sqlalchemy

In [2]:
msft = yf.Ticker("MSFT")

In [3]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [4]:
msft.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRati

In [5]:
print(msft.info["symbol"])
print(msft.info["longName"])
print(msft.info["country"])

MSFT
Microsoft Corporation
United States


In [9]:
config = configparser.ConfigParser()
config.read("config.ini")
user = config["local_postgres"]["DB_USER"]
pw = config["local_postgres"]["DB_PW"]
address = config["local_postgres"]["DB_ADDRESS"]
port = config["local_postgres"]["DB_PORT"]
name = config["local_postgres"]["DB_NAME"]

con_str = f"postgresql://{user}:{pw}@{address}:{port}/{name}"
con = sqlalchemy.create_engine(con_str)

In [11]:
query = """insert into finance.symbols values 
    ('MSFT', 'Microsoft', 'United States');"""
con.execute(query)

In [ ]:
symbol = msft.info["symbol"]
name = msft.info["longName"]
country = msft.info["country"]
query = f"""insert into finance.symbols values 
    ('{symbol}', '{name}', '{country}');"""
con.execute(query)

In [12]:
def insert_symbol(con, ticker):
    # Daten aus Ticker auslesen
    symbol = ticker.info["symbol"]
    name = ticker.info["longName"]
    country = ticker.info["country"]
    # Daten in Datenbank schreiben
    query = f"""insert into finance.symbols values 
        ('{symbol}', '{name}', '{country}');"""    
    con.execute(query)

t = yf.Ticker("AAPL")
insert_symbol(con, t)

In [13]:
insert_symbol(con, t)

IntegrityError: (psycopg2.errors.UniqueViolation) FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »symbols_pkey«
DETAIL:  Schlüssel »(symbol)=(AAPL)« existiert bereits.

[SQL: insert into finance.symbols values 
        ('AAPL', 'Apple Inc.', 'United States');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [17]:
symbol = 'MSFT'
query = f"""select count(1) from finance.symbols 
                where symbol = '{symbol}';"""
df = pd.read_sql(query,con)
df.iloc[0,0] == 1

True

In [20]:
def insert_symbol(con, ticker):
    # Daten aus Ticker auslesen
    symbol = ticker.info["symbol"]
    name = ticker.info["longName"]
    country = ticker.info["country"]
    # Prüfen, ob Symbol schon vorhanden
    query = f"""select count(1) from finance.symbols 
                where symbol = '{symbol}';"""
    df = pd.read_sql(query,con)
    vorhanden = df.iloc[0,0] == 1
    # Daten in Datenbank schreiben
    if not vorhanden:
        query = f"""insert into finance.symbols values 
            ('{symbol}', '{name}', '{country}');"""    
        con.execute(query)
    else:
        print("Eintrag existiert schon")

In [21]:
t = yf.Ticker("AAPL")
insert_symbol(con, t)

Eintrag existiert schon
